In [1]:
import pandas as pd
import numpy as np
import csv
import nltk
import xml.etree.ElementTree as ET
from nltk.util import everygrams

import re

In [2]:
# import parser
import spacy 
from spacy.matcher import PhraseMatcher 
nlp = spacy.load('en_core_web_sm')
matcher = PhraseMatcher(nlp.vocab)

# terms would be each list in the dict

In [3]:
import json 
# with open ('2020_covid_lexiconV1.json') as f1:
#     dic = json.load(f1)
    
with open ('2020_covid_lexiconV2.json') as f1:
    dic = json.load(f1)

In [4]:
print(sorted(dic))

['Covid']


In [36]:
# import json
# with open ('2019_public_comment.json','w') as f1:
#     json.dump(dic,f1)

In [5]:
from nltk.corpus import stopwords
stopword = stopwords.words('english')

In [6]:
df1 = pd.read_csv('twitter_posts_1000_05-30-2020.csv')
# df1 = pd.read_excel('Facebook Ads Analysis.xlsx') # this data is coming from the facebook ad 2016 (the )

In [7]:
df1.columns

Index(['Unnamed: 0', '_id', 'candidate_name', 'created_at', 'favorite_count',
       'reply_to', 'retweet_count', 'retweet_status', 'text',
       'tw_account_handle', 'tweet_id'],
      dtype='object')

In [8]:
df1 = df1[['text']]
df1['covid_label']=''

In [9]:
def filter_text(x):
    url = 'http[s]?://\S+'
    x = re.sub(url,'',x)
    x = re.sub("[^\w\s]",' ',x)
    x = re.sub("\s+",' ',x)
    
    ls=[w for w in x.split() if w.lower() not in stopword]
    
    return ' '.join(ls)

In [10]:
df1['text'] = df1['text'].apply(lambda x: filter_text(x))

In [11]:
df1

,text,covid_label
0,Sleepy Joe thought OK,
1,,
2,individual Governors States local officials mu...,
3,Thank Eli Lake,
4,Welcome Jezebel blogger BernieSanders promises...,
...,...,...
995,another clear warning sign needs done keep man...,
996,LIVE Press Briefing Coronavirus Task Force,
997,Newly Revealed Docs Support Flynn Innocence Ca...,
998,yet read single Democrat leader OUTRAGED explo...,


In [12]:
def find_words(x,lexicon):
    topics=['Covid']
    
    
    doc = nlp(x)
    words= []
    for t in topics:
        terms= lexicon[t]
        patterns = [nlp.make_doc(text) for text in terms]
        matcher.add("TerminologyList", None, *patterns)
        matches = matcher(doc)
        for match_id, start,end in matches:
            span = doc[start:end]
            words.append(span.text)
    words = list(set(words))
            
    return ','.join(words)
        

In [13]:
# find the words in each message
def find_topic(x,lexicon):
    topics=['Covid']
    
    if x=='':
        return ''
    
    if x != '': 
        words = x.split(',')
        labels = []
        
        for t in topics:            
            terms = lexicon[t]
            if set(words)&set(terms):
                #print(t)
                #print(set(words)&set(terms))
                labels.append(t)
                
                #l = sorted(labels)
        
        return  ','.join(sorted(labels))
                
        #return ','.join(labels)
            

In [14]:
# get the new labels from classifer2 which depending on the updated lexicon
df1['words'] = df1['text'].astype(str).apply(lambda x: find_words(x,dic))
#df1.rename(columns={'m_label':'words'},inplace=True)

In [15]:
df1.tail(10)

,text,covid_label,words
990,much time sleep Thank,,
991,get crisis supporting vulnerable identified ne...,,"pandemic,essential"
992,Emmieinthecity Michelle great idea Thank ensur...,,masks
993,Mueller investigation viewed exposure truth Ob...,,
994,Reports H H Secretary AlexAzar going fired Fak...,,
995,another clear warning sign needs done keep man...,,
996,LIVE Press Briefing Coronavirus Task Force,,Coronavirus
997,Newly Revealed Docs Support Flynn Innocence Ca...,,
998,yet read single Democrat leader OUTRAGED explo...,,
999,Looting protesting Burning local homes busines...,,


In [16]:
df1['covid_label'] = df1['words'].apply(lambda x: find_topic(x,dic))

In [17]:
df1['covid_label'] = df1['covid_label'].apply(lambda x: 'no topic' if x=='' else x)

In [18]:
df1.head(10)

,text,covid_label,words
0,Sleepy Joe thought OK,no topic,
1,,no topic,
2,individual Governors States local officials mu...,Covid,testing
3,Thank Eli Lake,no topic,
4,Welcome Jezebel blogger BernieSanders promises...,no topic,
5,extraordinary moment American world history go...,no topic,
6,Expand vote mail early voting Implement online...,no topic,
7,Today remember atrocities faced Armenian peopl...,no topic,
8,realDonaldTrump Correct 15 days keep high risk...,no topic,
9,South Korea continues show signs epidemic abat...,Covid,"infected patients,isolating,virus,testing"


In [19]:
df1.to_csv('2020may30_1000_tw_CovidLabel_V4.csv',index=False)